In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Input files
from google.colab import files
data = files.upload()

In [ ]:
# Assign files name
import io

dec_forecast = pd.read_excel(io.BytesIO(data['Dec_2022_Forecast & Consumption.xlsx']))

In [ ]:
dec_stock = pd.read_excel(io.BytesIO(data['Dec_2022_Stock Level.xlsx']))

### **Data Cleaning- December_Stock**

In [ ]:
dec_stock_copy = dec_stock.copy()

In [ ]:
# Convert the 'Date' column to datetime format
dec_stock_copy['Date'] = pd.to_datetime(dec_stock_copy['Date'])

# Create a new column with the combination of 'FC' and 'Item'
dec_stock_copy['FC_Item'] = dec_stock_copy['FC'] + '_' + dec_stock_copy['Item']

# Set the 'Date' column as the index
dec_stock_copy.set_index('Date', inplace=True)

In [ ]:
# Get unique values of 'FC_Item'
unique_items = dec_stock_copy['FC_Item'].unique()

# Create a date range from December 1st, 2022, to December 31st, 2022
dates = pd.date_range(start='2022-12-01', end='2022-12-31', freq='D')

# Repeat the unique items for each day in the date range
repeated_items = np.repeat(unique_items, len(dates))

# Repeat the date range for each unique item
repeated_dates = np.tile(dates, len(unique_items))

# Create a new DataFrame with the repeated values
new_df = pd.DataFrame({'FC_Item': repeated_items, 'Date': repeated_dates})

# Reset the index of the new DataFrame
new_df.set_index('Date', inplace=True)

In [ ]:
# Merge new_df with dec_stock_copy using 'FC_Item' and 'Date' as the keys
merged_df = pd.merge(dec_stock_copy, new_df, on=['FC_Item', 'Date'], how='outer')

In [ ]:
# Split 'FC_Item' column back into 'FC' and 'Item' columns
merged_df[['FC', 'Item']] = merged_df['FC_Item'].str.split('_', expand=True)

In [ ]:
# Sort the DataFrame by 'FC', 'Item', and 'Date' columns
merged_df.sort_values(by=['FC', 'Item', 'Date'], inplace=True)

In [ ]:
# Forward fill missing stock values within each 'FC_Item' group
merged_df['Stock'] = merged_df.groupby(['FC', 'Item'])['Stock'].fillna(method='ffill')

# Backward fill missing stock values within each 'FC_Item' group
merged_df['Stock'] = merged_df.groupby(['FC', 'Item'])['Stock'].fillna(method='bfill')

In [ ]:
# Save merged_df to an Excel file
merged_df.to_excel('Dec_Stock.xlsx', index=True)

##**Dec_Forecast**

In [ ]:
dec_forecast_copy = dec_forecast.copy()

In [ ]:
# Convert the 'Date' column to datetime format
dec_forecast_copy['Date'] = pd.to_datetime(dec_forecast_copy['Date'])

# Set the 'Date' column as the index
dec_forecast_copy.set_index('Date', inplace=True)

In [ ]:
# Define a function to compute the daily average values for each week
def compute_daily_averages(group):
    weekly_forecast = (group['Forecast'] / 7).round().astype(int)
    weekly_ordered = (group['Ordered'] / 7).round().astype(int)
    weekly_consumption = (group['Usage'] / 7).round().astype(int)

    # Calculate the start date of the first week (first Sunday of December)
    start_date = pd.to_datetime('2022-12-04')  # Adjust the start date based on the actual first Sunday
    
    # Repeat the weekly values for each day of the corresponding week
    repeated_forecast = np.repeat(weekly_forecast, 7)
    repeated_ordered = np.repeat(weekly_ordered, 7)
    repeated_consumption = np.repeat(weekly_consumption, 7)

    # Create a DataFrame with the repeated values for the FC-Item combination
    item_df = pd.DataFrame({
        'FC': group['FC'].iloc[0],
        'Item': group['Item'].iloc[0],
        'Date': pd.date_range(start=start_date, periods=len(repeated_forecast), freq='D'),
        'Forecast': repeated_forecast,
        'Consumption': repeated_consumption,
        'Ordered': repeated_ordered
    })

    return item_df

# Apply the function to each unique FC-Item combination
new_df = dec_forecast_copy.groupby(['FC', 'Item']).apply(compute_daily_averages).reset_index(drop=True)


In [ ]:
new_df

In [ ]:
# Save new_df to an Excel file
new_df.to_excel('Dec_Forecast.xlsx', index=True)